In [1]:
import itertools
from map_fsm import map_order_entity
import pandas as pd

In [2]:

list_define_target = []

for key in list(map_order_entity.keys()):

    if key != 'all_slot':

        list_slot = map_order_entity[key]
        list_permute = list(itertools.permutations(list_slot))
        list_permute_fix = [['initial',key] + list(item) for item in list_permute]
        
        for sublist_permute in list_permute_fix:
            
            for idx, slot in enumerate(sublist_permute):
                dict_target = {}
                if idx < len(sublist_permute) - 2:
                    dict_target['request'] = key
                    dict_target['input'] = sublist_permute[idx:idx+2]
                    dict_target['target'] = sublist_permute[idx+2]

                    if dict_target not in list_define_target:
                        list_define_target.append(dict_target)

In [3]:
df_define_target = pd.DataFrame(list_define_target)
df_define_target

,request,input,target
0,major_name,"[initial, major_name]",point
1,major_name,"[major_name, point]",subject_group
2,major_name,"[initial, major_name]",subject_group
3,major_name,"[major_name, subject_group]",point
4,major_code,"[initial, major_code]",major_name
...,...,...,...
174,case,"[initial, case]",major_name
175,case,"[case, major_name]",type_edu
176,case,"[initial, case]",type_edu
177,case,"[case, type_edu]",major_name


In [4]:
list_request = list(df_define_target['request'])
list_input = list(df_define_target['input'])
list_target = list(df_define_target['target'])

In [15]:
request_slot = ['major_name']
inform_slot = ['point', 'subject_group']
all_slot = request_slot + inform_slot
pattern_target = all_slot + map_order_entity[request_slot[0]].copy()

In [16]:
## update state tracker

list_state_tracker = ['initial']
list_state_tracker += all_slot


def recursion_find_best_way(list_state_tracker,pattern_target,request_slot):
    
    ## last state is 
#     last_state = list_state_tracker[-1]
    
    ## remove last state --> sucess rate
    print('update list state tracker',list_state_tracker)
    for item in list_state_tracker:
        if item != 'initial' and item in pattern_target:
#             if last_state in pattern_target:
            pattern_target.remove(item)
    
        
    if len(pattern_target) == 0:
        print('sucess')
        return True
    
    for idx in range(len(list_state_tracker)):
        if idx < len(list_state_tracker) - 1:
            window = list_state_tracker[idx:idx+2]
#             window = list_state_tracker[-2:]

            for idx, sublist in enumerate(list_input):
                if 'initial' not in window:
                    if sorted(window) == sorted(sublist) and request_slot[0] == list_request[idx]:

                        target_match = list_target[idx]
                        ## update state tracker
                        if target_match not in list_state_tracker:
                            list_state_tracker.append(target_match)
                            break
                else:
                    if window == sublist and request_slot[0] == list_request[idx]:
                        target_match = list_target[idx]
                        ## update state tracker
                        if target_match not in list_state_tracker:
                            list_state_tracker.append(target_match)
                            break
                    
    ## recursion
    recursion_find_best_way(list_state_tracker,pattern_target,request_slot)


In [17]:
recursion_find_best_way(list_state_tracker,pattern_target,request_slot)

update list state tracker ['initial', 'major_name', 'point', 'subject_group']
update list state tracker ['initial', 'major_name', 'point', 'subject_group']
sucess


In [18]:
list_state_tracker

['initial', 'major_name', 'point', 'subject_group']